In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [4]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [5]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データを分析することで、さまざまなビジネスインサイトを得られます。以下に、主な傾向と考察を示します。

### 1. カテゴリー別売上の傾向
- **食品**：比較的安価な商品が多いですが、売上数量が多いことから、消費者の需要が高いことが示唆されます。このカテゴリーでは、特にりんごや牛乳の売上が高いと考えられます。
- **衣服**：単価が高い商品（Tシャツやジーンズなど）が多く、数量がそれに応じて少ないものの、売上総額は高い傾向があります。トレンドやシーズンに応じて売上が変動しやすいカテゴリーです。
- **日用品**：このカテゴリーにはバスブラシといった商品の売上がありますが、他の2つのカテゴリーに比べて売上が低い可能性があります。需要促進策が必要かもしれません。

### 2. 時間的な傾向
- 売上日の情報から、月ごとの売上動向を分析することができます。特定の月に売上が高い場合、その時期のプロモーションやセールからの影響が考えられます。
- 例えば、年末や新年に向けての特別セールが、食品や衣服の購買を促すピーク要因として作用している可能性があります。

### 3. 原価との関係
- 商品の原価と単価から、利益率を計算することで、どの商品が利益に寄与しているかを分析できます。例えば、ジーンズやTシャツは高い利益率を持つ反面、売上数量自体は他の商品に比べて低い場合があるため、利益と売上のバランスを考慮する必要があります。
- 食品の売上数量が多い一方、原価が低いため、全体の利益へのインパクトは相対的に小さくなる可能性があります。

### 4. 売上数量と単価の関係
- 高単価の商品は販売数量が少ない場合が多いという一般的傾向が見られます。例えば、Tシャツやジーンズのような商品は、消費者が一度に購入する数量が少ないですが、単価が高いため、売上には寄与しています。
- この傾向をもとに、販売戦略を調整することが重要です。たとえば、高単価商品に対してバンドル販売や割引を提供する等の施策が効果的でしょう。

### 5. 提案
- **プロモーション戦略の見直し**：売上が低い商品やカテゴリーに対して特別なプロモーションを行うことを検討します。
- **商品ラインナップの最適化**：売上データをもとに、人気商品や利益率の高い商品の拡充を図るとともに、需要の低

In [6]:
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   この売上データを分析することで、さまざまなビジネスインサイトを得られます。以下に、主な傾向と...
1                                                    
2                                  ### 1. カテゴリー別売上の傾向
3   - **食品**：比較的安価な商品が多いですが、売上数量が多いことから、消費者の需要が高いこ...
4   - **衣服**：単価が高い商品（Tシャツやジーンズなど）が多く、数量がそれに応じて少ないも...
5   - **日用品**：このカテゴリーにはバスブラシといった商品の売上がありますが、他の2つのカ...
6                                                    
7                                       ### 2. 時間的な傾向
8   - 売上日の情報から、月ごとの売上動向を分析することができます。特定の月に売上が高い場合、そ...
9   - 例えば、年末や新年に向けての特別セールが、食品や衣服の購買を促すピーク要因として作用して...
10                                                   
11                                      ### 3. 原価との関係
12  - 商品の原価と単価から、利益率を計算することで、どの商品が利益に寄与しているかを分析できま...
13  - 食品の売上数量が多い一方、原価が低いため、全体の利益へのインパクトは相対的に小さくなる可...
14                                                   
15                                  ### 4. 売上数量と単価の関係
16  - 高単価の商品は販売数量が少ない場合が多いという一般的傾向が見られます。例えば、Tシャツや...
17  - この傾向をもとに、販売戦略を調整することが重

In [7]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [8]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
